In [15]:
# importamos las librerías necesarias y establecemos conexión
import sim          # librería para conectar con CoppeliaSim
import sys
import time
import numpy as np
from random import randint, uniform,random
from tensorflow.keras.models import load_model
from skimage.io import imsave
import cv2

def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

clientID = connect(19999)
# Controladores
retCode,Motor_D1=sim.simxGetObjectHandle(clientID,'Motor_D1',sim.simx_opmode_blocking)
retCode,Motor_D2=sim.simxGetObjectHandle(clientID,'Motor_D2',sim.simx_opmode_blocking)
retCode,Motor_D3=sim.simxGetObjectHandle(clientID,'Motor_D3',sim.simx_opmode_blocking)
retCode,Motor_I1=sim.simxGetObjectHandle(clientID,'Motor_I1',sim.simx_opmode_blocking)
retCode,Motor_I2=sim.simxGetObjectHandle(clientID,'Motor_I2',sim.simx_opmode_blocking)
retCode,Motor_I3=sim.simxGetObjectHandle(clientID,'Motor_I3',sim.simx_opmode_blocking)
retCode,psensor=sim.simxGetObjectHandle(clientID,'Psensor',sim.simx_opmode_blocking)
res,v0=sim.simxGetObjectHandle(clientID,'Vision_sensor',sim.simx_opmode_blocking)
# Controladores agua
retCode,G1=sim.simxGetObjectHandle(clientID,'Gotas_1',sim.simx_opmode_blocking)
retCode,G2=sim.simxGetObjectHandle(clientID,'Gotas_2',sim.simx_opmode_blocking)
retCode,G3=sim.simxGetObjectHandle(clientID,'Gotas_3',sim.simx_opmode_blocking)
retCode,G4=sim.simxGetObjectHandle(clientID,'Gotas_4',sim.simx_opmode_blocking)
retCode,G5=sim.simxGetObjectHandle(clientID,'Gotas_5',sim.simx_opmode_blocking)
retCode,S=sim.simxGetObjectHandle(clientID,'Salpicadura',sim.simx_opmode_blocking)
model = load_model("my_model.h5")
agua = 100
print( Motor_D1,Motor_D2,Motor_D3,Motor_I1,Motor_I2,Motor_I3,psensor )

conectado a 19999
26 17 23 29 32 35 19


In [16]:
errorCode, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector=sim.simxReadProximitySensor(clientID,psensor, sim.simx_opmode_blocking)
# podemos calcular la distancia con  #simx_opmode_buffer
if detectionState==True:
    sensor_val=np.linalg.norm(detectedPoint)
    print("distacia al objeto: ",sensor_val)
else:
    print("No ha detectado objeto")

No ha detectado objeto


In [17]:
def regado(times):
    for veces in range(times):
        sim.simxSetObjectIntParameter(clientID,G1,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,G1,10,0,sim.simx_opmode_oneshot_wait)
        sim.simxSetObjectIntParameter(clientID,G2,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,G2,10,0,sim.simx_opmode_oneshot_wait)
        sim.simxSetObjectIntParameter(clientID,G3,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,G3,10,0,sim.simx_opmode_oneshot_wait)
        sim.simxSetObjectIntParameter(clientID,G4,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,G4,10,0,sim.simx_opmode_oneshot_wait)
        sim.simxSetObjectIntParameter(clientID,G5,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,G5,10,0,sim.simx_opmode_oneshot_wait)
        sim.simxSetObjectIntParameter(clientID,S,10,5,sim.simx_opmode_oneshot_wait)
        time.sleep(0.025)
        sim.simxSetObjectIntParameter(clientID,S,10,0,sim.simx_opmode_oneshot_wait)

In [18]:
time.sleep(0.5)
t = time.time()
vd = 1
vi = 1
i = 0
count_margarita=0
count_dandelion=0
count_rosas=0
count_girasol=0
count_tulipan=0

#Poner toda la lluvia en invisible
sim.simxSetObjectIntParameter(clientID,G1,10,0,sim.simx_opmode_oneshot_wait)
sim.simxSetObjectIntParameter(clientID,G2,10,0,sim.simx_opmode_oneshot_wait)
sim.simxSetObjectIntParameter(clientID,G3,10,0,sim.simx_opmode_oneshot_wait)
sim.simxSetObjectIntParameter(clientID,G4,10,0,sim.simx_opmode_oneshot_wait)
sim.simxSetObjectIntParameter(clientID,G5,10,0,sim.simx_opmode_oneshot_wait)
sim.simxSetObjectIntParameter(clientID,S,10,0,sim.simx_opmode_oneshot_wait)

#Avanzar hacia delante durante 2 segundos:
sim.simxSetJointTargetVelocity(clientID, Motor_D1,vd,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Motor_D2,vd,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Motor_D3,vd,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Motor_I1,vi,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Motor_I2,vi,sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Motor_I3,vi,sim.simx_opmode_streaming)

while agua > 0:
    
        errorCode, detectionState, detectedPoint, detectedObjectHandle, detectedSurfaceNormalVector=sim.simxReadProximitySensor(clientID,psensor, sim.simx_opmode_blocking)

      

        sensor_val = np.linalg.norm(detectedPoint)
        if sensor_val < 0.73 and sensor_val>0.05:

            vd = 0
            vi = 0

            sim.simxSetJointTargetVelocity(clientID, Motor_D1,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_D2,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_D3,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I1,vi,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I2,vi,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I3,vi,sim.simx_opmode_streaming)
            
            
            err, resolution, image = sim.simxGetVisionSensorImage(clientID, v0, 0, sim.simx_opmode_streaming)
            while err != 0:
                err, resolution, image = sim.simxGetVisionSensorImage(clientID, v0, 0, sim.simx_opmode_streaming)
            
            im = np.array(image);
            im = np.resize(im,(150,150,3))
            im = np.rot90(im, k=2, axes=(0, 1))
            imsave(r"prueba.jpg", abs(im))
            img = cv2.imread("prueba.jpg",cv2.IMREAD_COLOR)
            #If (Hay planta)
            val = np.argmax(model.predict(np.array([img/255])),axis=1)
            
            if val < 5:
                
                    if val == 0: 
                    #If Margarita (se riega 1 veces)
                        if count_margarita<1:
                            count_margarita=1
                            agua = agua - 1
                            regado(1)
                            
                    if val == 1:
                    #If Diente de leon (se riega 3 veces)
                        if count_dandelion<1:
                            count_dandelion=1
                            agua = agua - 3
                            regado(3)
                    
                    if val == 2:
                    #If Tulipan (se riega 2 veces)
                        if count_tulipan<1:
                            count_tulipan=1
                            agua = agua - 2
                            regado(2)
                    
                    if val == 3:
                    #If Rosa (se reiga 4 veces)
                        if count_rosas<1:
                            count_rosas=1
                            agua = agua - 4
                            regado(4)
                    
                    if val == 4:
                    #If Girasol (se riega 5 veces)
                        if count_girasol<1:
                            count_girasol=1
                            agua = agua - 5
                            regado(5)
                           

            time.sleep(2)

            vd = 1
            vi = 1
            
            val = randint(1,100)
            tiempo = randint(1,3)
            
            if (val < 50 ):

                
                sim.simxSetJointTargetVelocity(clientID, Motor_D1,vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_D2,vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_D3,vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I1,-vi,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I2,-vi,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I3,-vi,sim.simx_opmode_streaming)
                time.sleep(tiempo)
                
            else: 
                sim.simxSetJointTargetVelocity(clientID, Motor_D1,-vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_D2,-vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_D3,-vd,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I1,vi,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I2,vi,sim.simx_opmode_streaming)
                sim.simxSetJointTargetVelocity(clientID, Motor_I3,vi,sim.simx_opmode_streaming)
                time.sleep(tiempo)
        else:
            vd = 1
            vi = 1
            time.sleep(0.2)
            sim.simxSetJointTargetVelocity(clientID, Motor_D1,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_D2,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_D3,vd,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I1,vi,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I2,vi,sim.simx_opmode_streaming)
            sim.simxSetJointTargetVelocity(clientID, Motor_I3,vi,sim.simx_opmode_streaming)


<ipython-input-18-3ded7d1a9295>:55: UserWarning: prueba.jpg is a low contrast image
  imsave(r"prueba.jpg", abs(im))
Lossy conversion from int32 to uint8. Range [0, 122]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 116]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 124]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 117]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int32 to uint8. Range [0, 118]. Convert image to uint8 prior to saving to suppress this warning.


KeyboardInterrupt: 